In [1]:
from dotabet.ui import send_telegram_message
from dotabet.script.pinnacle import scrape_dota2_odds   
from dotabet.script.recommend import recommend_elo  

parse_token = "7416284654:AAGhRdKNgUlwDrBRPp_xRaMO151aMMh11jY"
chat_id = "333091706"


tg_msg = scrape_dota2_odds()
send_telegram_message(parse_token, chat_id, ''.join(tg_msg))

# dota2_rec_bot
rec_token = "7227209457:AAHHtQ2mSnHyK6zzyIjQZFyrLd4CXBZn3_o"
tg_msg_rec = recommend_elo()  
send_telegram_message(rec_token, chat_id, ''.join(tg_msg_rec))

Failed to send message.
Message sent successfully.


In [2]:
from dotabet.script.pinnacle import *

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

# Path to the ChromeDriver executable
chrome_driver_path = r"D:\PROGRAMS\chromedriver-win64\chromedriver.exe" # Update this to your path

# Setting up Chrome options
chrome_options = Options()
chrome_options.add_argument("--start-maximized")  # Start browser maximized
chrome_options.add_argument("--disable-infobars")  # Disables the "Chrome is being controlled by automated software" info bar
chrome_options.add_argument("--disable-extensions")  # Disables extensions
chrome_options.add_argument("--disable-gpu")  # Applicable to Windows OS only
chrome_options.add_argument("--no-sandbox")  # Bypass OS security model
chrome_options.add_argument("--headless")  # Run browser in headless mode (without GUI)

# Define the Chrome driver
driver = webdriver.Chrome(service=Service(chrome_driver_path), options=chrome_options)

# Example: Navigate to a website
driver.get("https://www.google.com")

# Close the browser
driver.quit()


In [3]:
telegram_msg = []
odds_file = r"D:\WORKSPACE\dotabet\data\odds\pinnacle.csv"


driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://www.pinnacle.com/en/esports/games/dota-2/matchups")
time.sleep(5) 

square = driver.find_element(By.CSS_SELECTOR, "div.contentBlock.square > div.contentBlock.square")
content_blocks = square.find_elements(By.CSS_SELECTOR, "div[class^='style_']")

attributes = ['style_dateBar__1adEH', 'style_row__yBzX8 style_row__3l5MS', 'style_row__yBzX8 style_row__12oAB']
content_blocks = [cb for cb in content_blocks if cb.get_attribute("class") in attributes]
n = len(content_blocks)

for i, block in enumerate(content_blocks):
    if block.get_attribute("class") == 'style_dateBar__1adEH': # TODAY, TOMMOROW or DATE
        date_ymd = get_date_ymd(block)
        print(f"block[{i}/{n}]=Match_date: {date_ymd}")
        continue
    elif block.get_attribute("class") == "style_row__yBzX8 style_row__3l5MS": # LEAGUE NAME
        league_name = block.find_element(By.CSS_SELECTOR, "div.style_metadata__3MrIC > a > span").text
        print(f"block[{i}/{n}]=League_name: {league_name}")
        continue
    elif block.get_attribute("class") == "style_row__yBzX8 style_row__12oAB": # MATCH ODDS
        odd_data = get_match_odds(block)
        if odd_data and odd_data[1]: # (Match) or (Map 1) and non-empty
            data = [date_ymd, league_name] + odd_data + [None, None] # None for eloc1, eloc2 to pinnacle.csv
            if append_to_csv(odds_file, data):
                telegram_msg.append(f"\n✅New:{data}")
            else:
                telegram_msg.append("🔄")
            print(f"block[{i}/{n}]=Match_odds: {data=}")
        elif odd_data and not odd_data[1]: # (Match) but empty odds
            print(f"block[{i}/{n}] = (Match) but blocked odds🚫")
            telegram_msg.append("🚫")
        else:
            print(f"block[{i}/{n}]=<Skip non-(Match)> ")
    else:
        unknown_block = block.get_attribute("class")
        print(f"block {i}/{n} Unknown {unknown_block}")
driver.quit()

OSError: [WinError 193] %1 is not a valid Win32 application